In [1]:
import os
import argparse
import datetime
import tensorflow as tf
import yolo.config as cfg
from yolo.yolo_net import YOLONet
from utils.timer import Timer
from utils.pascal_voc import pascal_voc
import easydict
import time
slim = tf.contrib.slim
print("ok")

Instructions for updating:
Use the retry module or similar alternatives.
ok


In [2]:

class Solver(object):

    def __init__(self, net, data):
        print("Solver init")
        self.net = net
        self.data = data
        self.weights_file = cfg.WEIGHTS_FILE
        self.max_iter = cfg.MAX_ITER
        self.initial_learning_rate = cfg.LEARNING_RATE
        self.decay_steps = cfg.DECAY_STEPS
        self.decay_rate = cfg.DECAY_RATE
        self.staircase = cfg.STAIRCASE
        self.summary_iter = cfg.SUMMARY_ITER
        self.save_iter = cfg.SAVE_ITER
        self.output_dir = os.path.join(cfg.OUTPUT_DIR, datetime.datetime.now().strftime('%Y_%m_%d_%H_%M'))
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
        self.save_cfg()

        self.variable_to_restore = tf.global_variables()
        self.saver = tf.train.Saver(self.variable_to_restore, max_to_keep=None)
        self.ckpt_file = os.path.join(self.output_dir, 'yolo')
        self.summary_op = tf.summary.merge_all()
        self.writer = tf.summary.FileWriter(self.output_dir, flush_secs=60)

        self.global_step = tf.train.create_global_step()
        self.learning_rate = tf.train.exponential_decay(
            self.initial_learning_rate, self.global_step, self.decay_steps,
            self.decay_rate, self.staircase, name='learning_rate')
        self.optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=self.learning_rate)
        self.train_op = slim.learning.create_train_op(
            self.net.total_loss, self.optimizer, global_step=self.global_step)

        gpu_options = tf.GPUOptions()
        config = tf.ConfigProto(gpu_options=gpu_options)
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())

        if self.weights_file is not None:
            print('Restoring weights from: ' + self.weights_file)
            self.saver.restore(self.sess, self.weights_file)

        self.writer.add_graph(self.sess.graph)
        print("Solver init done!")

    def train(self):
        print("Solver train() start")

        train_timer = Timer()
        load_timer = Timer()

        for step in range(1, self.max_iter + 1):
            print("step:%d"%step)
            time.sleep(5)
            load_timer.tic()
            images, labels = self.data.get()
            load_timer.toc()
            feed_dict = {self.net.images: images,
                         self.net.labels: labels}

            if step % self.summary_iter == 0:
                if step % (self.summary_iter * 10) == 0:

                    train_timer.tic()
                    summary_str, loss, _ = self.sess.run(
                        [self.summary_op, self.net.total_loss, self.train_op],
                        feed_dict=feed_dict)
                    train_timer.toc()

                    log_str = '''{} Epoch: {}, Step: {}, Learning rate: {},'''
                    ''' Loss: {:5.3f}\nSpeed: {:.3f}s/iter,'''
                    '''' Load: {:.3f}s/iter, Remain: {}'''.format(
                        datetime.datetime.now().strftime('%m-%d %H:%M:%S'),
                        self.data.epoch,
                        int(step),
                        round(self.learning_rate.eval(session=self.sess), 6),
                        loss,
                        train_timer.average_time,
                        load_timer.average_time,
                        train_timer.remain(step, self.max_iter))
                    print(log_str)

                else:
                    train_timer.tic()
                    summary_str, _ = self.sess.run(
                        [self.summary_op, self.train_op],
                        feed_dict=feed_dict)
                    train_timer.toc()

                self.writer.add_summary(summary_str, step)

            else:
                train_timer.tic()
                self.sess.run(self.train_op, feed_dict=feed_dict)
                train_timer.toc()

            if step % self.save_iter == 0:
                print('{} Saving checkpoint file to: {}'.format(
                    datetime.datetime.now().strftime('%m-%d %H:%M:%S'),
                    self.output_dir))
                self.saver.save(
                    self.sess, self.ckpt_file, global_step=self.global_step)

    def save_cfg(self):

        with open(os.path.join(self.output_dir, 'config.txt'), 'w') as f:
            cfg_dict = cfg.__dict__
            for key in sorted(cfg_dict.keys()):
                if key[0].isupper():
                    cfg_str = '{}: {}\n'.format(key, cfg_dict[key])
                    f.write(cfg_str)



In [3]:

def update_config_paths(data_dir, weights_file):
    cfg.DATA_PATH = data_dir
    cfg.PASCAL_PATH = os.path.join(data_dir, 'pascal_voc')
    cfg.CACHE_PATH = os.path.join(cfg.PASCAL_PATH, 'cache')
    cfg.OUTPUT_DIR = os.path.join(cfg.PASCAL_PATH, 'output')
    cfg.WEIGHTS_DIR = os.path.join(cfg.PASCAL_PATH, 'weights')
    #cfg.WEIGHTS_DIR = "data/weights"

    cfg.WEIGHTS_FILE = os.path.join(cfg.WEIGHTS_DIR, weights_file)
    #cfg.WEIGHTS_FILE = "data/weights/YOLO_small．ckpt"

print("ok")

ok


In [4]:

def main():
    '''
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', default="YOLO_small.ckpt", type=str)
    parser.add_argument('--data_dir', default="data", type=str)
    parser.add_argument('--threshold', default=0.2, type=float)
    parser.add_argument('--iou_threshold', default=0.5, type=float)
    parser.add_argument('--gpu', default='', type=str)
    args = parser.parse_args()
    args = easydict.EasyDict( {
           "weights":"YOLO_small.ckpt",
           "data_dir":"data",
           "threshold":0.2,
           "gpu":"",
           "iou_threshold":0.5
    })
    
    if args.gpu is not None:
        cfg.GPU = args.gpu

    if args.data_dir != cfg.DATA_PATH:
        update_config_paths(args.data_dir, args.weights)

    os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU
    ''' 
    yolo = YOLONet()
    print("build yolo model done")
    pascal = pascal_voc('train')

    solver = Solver(yolo, pascal)

    print('Start training ...')
    #solver.train()
    print("假装已经训练玩啦")
    print('Done training.')
    
print("ok")

# python train.py --weights YOLO_small.ckpt --gpu 0   

ok


In [5]:
 main()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
build yolo model done
Processing gt_labels from: data/pascal_voc/VOCdevkit/VOC2007
image_index is:000005
image_index is:000007
image_index is:000009
image_index is:000012
image_index is:000016
image_index is:000017
image_index is:000019
image_index is:000020
image_index is:000021
image_index is:000023
image_index is:000024
image_index is:000026
image_index is:000030
image_index is:000032
image_index is:000033
image_index is:000034
image_index is:000035
image_index is:000036
image_index is:000039
image_index is:000041
image_index is:000042
image_index is:000044
image_index is:000046
image_index is:000047
image_index is:000048
image_index is:000050
image_index is:000051
image_index is:000052
image_index is:000060
image_index is:000061
image_index is:000063
image_index is:000064
image_index is:000065
image_index is:000066
image_index is:000072
image_index is:000073
image_index is:000077
image_index is:000078
image_in

image_index is:000756
image_index is:000760
image_index is:000761
image_index is:000763
image_index is:000764
image_index is:000767
image_index is:000768
image_index is:000770
image_index is:000771
image_index is:000772
image_index is:000774
image_index is:000776
image_index is:000777
image_index is:000780
image_index is:000782
image_index is:000786
image_index is:000787
image_index is:000791
image_index is:000793
image_index is:000794
image_index is:000796
image_index is:000797
image_index is:000799
image_index is:000800
image_index is:000802
image_index is:000804
image_index is:000805
image_index is:000806
image_index is:000808
image_index is:000810
image_index is:000812
image_index is:000814
image_index is:000815
image_index is:000816
image_index is:000818
image_index is:000820
image_index is:000822
image_index is:000823
image_index is:000826
image_index is:000827
image_index is:000828
image_index is:000829
image_index is:000830
image_index is:000831
image_index is:000832
image_inde

image_index is:001504
image_index is:001509
image_index is:001510
image_index is:001512
image_index is:001514
image_index is:001515
image_index is:001517
image_index is:001521
image_index is:001522
image_index is:001523
image_index is:001524
image_index is:001526
image_index is:001528
image_index is:001529
image_index is:001531
image_index is:001532
image_index is:001536
image_index is:001537
image_index is:001539
image_index is:001541
image_index is:001543
image_index is:001544
image_index is:001545
image_index is:001548
image_index is:001553
image_index is:001554
image_index is:001555
image_index is:001556
image_index is:001557
image_index is:001559
image_index is:001561
image_index is:001563
image_index is:001565
image_index is:001571
image_index is:001576
image_index is:001577
image_index is:001579
image_index is:001580
image_index is:001582
image_index is:001586
image_index is:001588
image_index is:001590
image_index is:001593
image_index is:001594
image_index is:001595
image_inde

image_index is:002272
image_index is:002273
image_index is:002276
image_index is:002277
image_index is:002278
image_index is:002279
image_index is:002280
image_index is:002281
image_index is:002284
image_index is:002285
image_index is:002287
image_index is:002288
image_index is:002290
image_index is:002291
image_index is:002293
image_index is:002300
image_index is:002302
image_index is:002305
image_index is:002306
image_index is:002307
image_index is:002308
image_index is:002310
image_index is:002311
image_index is:002315
image_index is:002318
image_index is:002320
image_index is:002321
image_index is:002323
image_index is:002324
image_index is:002328
image_index is:002329
image_index is:002330
image_index is:002332
image_index is:002333
image_index is:002334
image_index is:002335
image_index is:002337
image_index is:002340
image_index is:002342
image_index is:002343
image_index is:002345
image_index is:002347
image_index is:002348
image_index is:002350
image_index is:002352
image_inde

image_index is:003011
image_index is:003013
image_index is:003015
image_index is:003017
image_index is:003021
image_index is:003023
image_index is:003024
image_index is:003027
image_index is:003028
image_index is:003031
image_index is:003032
image_index is:003034
image_index is:003038
image_index is:003039
image_index is:003042
image_index is:003044
image_index is:003045
image_index is:003047
image_index is:003051
image_index is:003053
image_index is:003054
image_index is:003056
image_index is:003057
image_index is:003058
image_index is:003061
image_index is:003063
image_index is:003064
image_index is:003065
image_index is:003066
image_index is:003072
image_index is:003074
image_index is:003077
image_index is:003078
image_index is:003082
image_index is:003083
image_index is:003085
image_index is:003086
image_index is:003088
image_index is:003089
image_index is:003090
image_index is:003092
image_index is:003093
image_index is:003094
image_index is:003098
image_index is:003100
image_inde

...

image_index is:008386
image_index is:008387
image_index is:008388
image_index is:008390
image_index is:008391
image_index is:008397
image_index is:008398
image_index is:008403
image_index is:008409
image_index is:008410
image_index is:008413
image_index is:008415
image_index is:008416
image_index is:008422
image_index is:008423
image_index is:008424
image_index is:008425
image_index is:008426
image_index is:008427
image_index is:008429
image_index is:008430
image_index is:008433
image_index is:008434
image_index is:008437
image_index is:008438
image_index is:008442
image_index is:008443
image_index is:008444
image_index is:008445
image_index is:008449
image_index is:008450
image_index is:008452
image_index is:008453
image_index is:008454
image_index is:008456
image_index is:008461
image_index is:008462
image_index is:008465
image_index is:008466
image_index is:008467
image_index is:008468
image_index is:008470
image_index is:008472
image_index is:008475
image_index is:008477
image_inde

image_index is:009205
image_index is:009208
image_index is:009209
image_index is:009212
image_index is:009213
image_index is:009214
image_index is:009215
image_index is:009218
image_index is:009221
image_index is:009224
image_index is:009227
image_index is:009230
image_index is:009236
image_index is:009238
image_index is:009239
image_index is:009242
image_index is:009244
image_index is:009245
image_index is:009246
image_index is:009247
image_index is:009249
image_index is:009250
image_index is:009251
image_index is:009252
image_index is:009254
image_index is:009255
image_index is:009259
image_index is:009268
image_index is:009269
image_index is:009270
image_index is:009271
image_index is:009272
image_index is:009273
image_index is:009278
image_index is:009279
image_index is:009281
image_index is:009282
image_index is:009283
image_index is:009285
image_index is:009286
image_index is:009287
image_index is:009288
image_index is:009289
image_index is:009290
image_index is:009291
image_inde

<class 'list'>


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Solver init
Solver init done!
Start training ...
假装已经训练玩啦
Done training.
